In [99]:
import pandas as pd

In [100]:
# Load the dataset
user = "aldi"  # Replace with the actual username
file_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/combined_davis.xlsx"  
df = pd.read_excel(file_path)

In [101]:
# Loop through the DataFrame and mark rows for deletion
to_delete = []
for i in range(len(df) - 1):
    # Check for consecutive rows with "Sam Groth" as "Player 1" and "John Peers" as "Player 2"
    if (df.iloc[i]['Player 1'] == "Sam Groth" and 
        df.iloc[i]['Player 2'] == "John Peers" and
        df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1'] and
        df.iloc[i]['Player 2'] == df.iloc[i + 1]['Player 2']):
        to_delete.append(i + 1)
    
    # Check for consecutive rows with "Jordan Thompson" as "Player 1"
    elif (df.iloc[i]['Player 1'] == "Jordan Thompson" and 
          df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1']):
        to_delete.append(i + 1)

    # Check for consecutive rows with "Nick Kyrgios" as "Player 1"
    elif (df.iloc[i]['Player 1'] == "Nick Kyrgios" and 
          df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1']):
        to_delete.append(i + 1)
        
    # Check for consecutive rows with "Sam Groth" as "Player 1"
    elif (df.iloc[i]['Player 1'] == "Sam Groth" and 
          df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1']):
        to_delete.append(i + 1)
    # Check for consecutive rows with "Sam Groth" as "Player 1" and "John Peers" as "Player 2"
    elif (df.iloc[i]['Player 1'] == "John Peers" and 
        df.iloc[i]['Player 2'] == "Jordan Thompson" and
        df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1'] and
        df.iloc[i]['Player 2'] == df.iloc[i + 1]['Player 2']):
        to_delete.append(i + 1)


# Remove the marked rows
df_cleaned = df.drop(df.index[to_delete])


In [102]:
# Extract the relevant columns and rename them
df_extracted = df_cleaned[['Stage', 'Player 1', 'Player 2', 'Year', 'Date', 'Venue', 'Court Pace Rating', 'Ball', 'match']].copy()
df_extracted.rename(columns={'Player 1': 'P1T1', 'Player 2': 'P2T1'}, inplace=True)

# Helper function to pair rows with the additional condition that "Date" should be equal
def pair_rows(df):
    paired_rows = []
    for i in range(0, len(df), 2):
        if i+1 < len(df):
            row1 = df.iloc[i]
            row2 = df.iloc[i+1]
            if row1['match'] == row2['match'] and row1['Year'] == row2['Year'] and row1['Date'] == row2['Date']:
                paired_row = [row1['match'], row1['Year'], row1['P1T1'], row1['P2T1'], row2['P1T1'], row2['P2T1'], row1['Date'], row1['Venue'], row1['Court Pace Rating'], row1['Ball']]
                paired_rows.append(paired_row)
    paired_df = pd.DataFrame(paired_rows, columns=['Match', 'Year', 'P1T1', 'P2T1', 'P1T2', 'P2T2', 'Date', 'Venue', 'Court Pace Rating', 'Ball'])
    return paired_df

# Pair the rows in the extracted dataframe
df_paired = pair_rows(df_extracted)

# Replace specific player names to match with data from atp matches
df_paired.replace({
    'P1T1': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
        'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'
    },
    'P2T1': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
        'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'

    },
    'P1T2': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
         'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'

    },
    'P2T2': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos',
        'Felix Auger-Aliassime': 'Felix Auger Aliassime',
         'Pierre-Hugues Herbert': 'Pierre Hugues Herbert',
        'Marc-Andrea Huesler': 'Marc Andrea Huesler',
        'Roman Khassanov': 'Roman Hassanov'

    }
}, inplace=True)


# Print the shapes of the dataframes
print("df shape:", df.shape)
print("df_cleaned shape:", df_cleaned.shape)
print("df_paired shape:", df_paired.shape)


df shape: (866, 28)
df_cleaned shape: (846, 28)
df_paired shape: (423, 10)


In [103]:
# Group by Date, Year, and Venue, then count observations
grouped_counts = df_cleaned.groupby(['Date', 'Year', 'Venue']).size()

# Define a function to check if counts meet the specified criteria
def check_count(year, count):
    if year <= 2018:
        return count == 8 or count == 10
    else:
        return 4 <= count <= 16

# Apply the function to each row in the grouped data and create a new column 'Count'
correct_counts_df = grouped_counts.reset_index()
correct_counts_df['Count'] = correct_counts_df.apply(lambda row: row[0], axis=1)

# Filter rows where counts do not meet the criteria
incorrect_counts_df = correct_counts_df[~correct_counts_df.apply(lambda row: check_count(row['Year'], row['Count']), axis=1)]

# Check if the DataFrame is empty, indicating all matches are in the correct range
if incorrect_counts_df.empty:
    print("All matches are in the correct range.")
else:
    # Display Date, Year, Venue, and Count when correct_counts is False
    print(incorrect_counts_df[['Date', 'Year', 'Venue', 'Count']])


All matches are in the correct range.


In [104]:
# keep only rows for single matches
davis_single_df = df_paired[df_paired['P2T1'].isna() & df_paired['P2T2'].isna()]
print("davis_single_df", davis_single_df.shape)

davis_single_df (331, 10)


In [105]:
davis_single_df.loc[davis_single_df['P1T2'] == 'Juan Martin del Potro',]

,Match,Year,P1T1,P2T1,P1T2,P2T2,Date,Venue,Court Pace Rating,Ball
195,MATCH 1,2016,Andy Murray,NaN,Juan Martin del Potro,NaN,16 Sep - 18 Sep 2016,"Emirates Arena, Glasgow, Great Britain",Medium Fast,Slazenger Wimbledon
206,MATCH 2,2016,Ivo Karlovic,NaN,Juan Martin del Potro,NaN,25 Nov - 27 Nov 2016,"Arena Zagreb, Zagreb, Croatia",Medium Slow,Wilson US Open Extra Duty
208,MATCH 4,2016,Marin Cilic,NaN,Juan Martin del Potro,NaN,25 Nov - 27 Nov 2016,"Arena Zagreb, Zagreb, Croatia",Medium Slow,Wilson US Open Extra Duty


In [106]:
folder_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/matches/"

# Initialize DataFrame to store all ATP matches where Davis Cup players took part
combined_matches_df = pd.DataFrame()

for year in range(2014, 2024):
    # Filter Davis Cup data for the current year
    davis_single_year_df = df_paired[df_paired['Year'] == year]

    # Create a set of tuples representing pairs of players for the current year
    davis_pairs = set(zip(davis_single_year_df['P1T1'], davis_single_year_df['P1T2']))

    # Load ATP matches data for the current year
    atp_matches_path = f"{folder_path}atp_matches_{year}.csv"
    atp_matches_df = pd.read_csv(atp_matches_path)

    # Filter ATP matches data to match Davis Cup pairs
    matched_atp_matches_df = atp_matches_df[
        atp_matches_df.apply(lambda row: (row['winner_name'], row['loser_name']) in davis_pairs or
                                           (row['loser_name'], row['winner_name']) in davis_pairs, axis=1)
    ]

    # Append the filtered data to the combined DataFrame
    combined_matches_df = combined_matches_df.append(matched_atp_matches_df)

# # Extract the year information before the first hyphen in 'tourney_id'
combined_matches_df['year'] = combined_matches_df['tourney_id'].str.split('-').str[0]

# Convert 'year' column to integer
combined_matches_df['year'] = combined_matches_df['year'].astype(int)

# Save the combined data to a single Excel file
final_output_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/matches/filtered_atp_matches.xlsx"
combined_matches_df.to_excel(final_output_path, index=False)


In [107]:
# Count the number of observations where tourney_level is "D" - Davis Cup
count_tourney_level_D = combined_matches_df[combined_matches_df['tourney_level'] == 'D'].shape[0]

# Count the number of observations where tourney_level is not "D"
count_not_tourney_level_D = combined_matches_df[combined_matches_df['tourney_level'] != 'D'].shape[0]

print("Number of observations with tourney_level 'D':", count_tourney_level_D)
print("Number of observations without tourney_level 'D':", count_not_tourney_level_D)


Number of observations with tourney_level 'D': 331
Number of observations without tourney_level 'D': 117


In [108]:
# Creating a new DataFrame with observations where tourney_level is "D"
davis_cup_matches_df = combined_matches_df[combined_matches_df['tourney_level'] == 'D']


In [109]:

# Creating a list with year, match_num, winner_name, loser_name, where names are alphabetically ordered
matches_list = davis_cup_matches_df.apply(
    lambda row: [row['year'], *sorted([row['winner_name'], row['loser_name']])], 
    axis=1
).tolist()

# Optionally, print the first few elements in the list to verify
print(matches_list[:5])


[[2014, 'Andreas Seppi', 'Carlos Berlocq'], [2014, 'Fabio Fognini', 'Juan Monaco'], [2014, 'Carlos Berlocq', 'Fabio Fognini'], [2014, 'Radek Stepanek', 'Robin Haase'], [2014, 'Igor Sijsling', 'Tomas Berdych']]


In [110]:
# Clean the 'Match' column to keep only the number after "MATCH"
davis_single_df['Match'] = davis_single_df['Match'].str.replace('MATCH ', '').astype(int)

# Creating a list with Year, cleaned Match, P1T1, P1T2, where player names are alphabetically ordered
davis_single_list = davis_single_df.apply(
    lambda row: [row['Year'], *sorted([row['P1T1'], row['P1T2']])], 
    axis=1
).tolist()

# Optionally, print the first few elements in the list to verify
print(davis_single_list[:5])


[[2014, 'Radek Stepanek', 'Robin Haase'], [2014, 'Igor Sijsling', 'Tomas Berdych'], [2014, 'Thiemo De Bakker', 'Tomas Berdych'], [2014, 'Igor Sijsling', 'Lukas Rosol'], [2014, 'Kei Nishikori', 'Peter Polansky']]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [111]:
# Standardizing formats in matches_list and davis_single_list
# Assuming year and match number are integers, and player names are strings

standardized_matches_list = [
    [int(year), str(winner_name), str(loser_name)]
    for year, winner_name, loser_name in matches_list
]

standardized_davis_single_list = [
    [int(year), str(player1), str(player2)]
    for year, player1, player2 in davis_single_list
]

# Convert to sets of tuples for comparison
standardized_matches_set = set(tuple(item) for item in standardized_matches_list)
standardized_davis_single_set = set(tuple(item) for item in standardized_davis_single_list)

# Find non-matching elements in both sets
non_matching_in_standardized_matches = standardized_matches_set - standardized_davis_single_set
non_matching_in_standardized_davis_single = standardized_davis_single_set - standardized_matches_set

print("Non-matching elements in standardized_matches_list:", non_matching_in_standardized_matches)
print("Non-matching elements in standardized_davis_single_list:", non_matching_in_standardized_davis_single)

# Count the number of non-matching tuples in standardized_matches_list
num_unmatched_in_standardized_matches = len(non_matching_in_standardized_matches)

# Count the number of non-matching tuples in standardized_davis_single_list
num_unmatched_in_standardized_davis_single = len(non_matching_in_standardized_davis_single)

print("Number of unmatched elements in standardized_matches_list:", num_unmatched_in_standardized_matches)
print("Number of unmatched elements in standardized_davis_single_list:", num_unmatched_in_standardized_davis_single)


Non-matching elements in standardized_matches_list: set()
Non-matching elements in standardized_davis_single_list: set()
Number of unmatched elements in standardized_matches_list: 0
Number of unmatched elements in standardized_davis_single_list: 0
